In [1]:
!pip install torchdata PyTDC rdkit-pypi transformers

In [2]:
from tdc.single_pred import ADME
from rdkit import Chem
from rdkit.Chem import AllChem
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [3]:
# 데이터 로드
split = 'scaffold'
data = ADME(name='CYP2C9_Veith')
split_data = data.get_split(method=split)

train_data, valid_data, test_data = split_data['train'], split_data['valid'], split_data['test']

Found local copy...
Loading...
Done!
100%|██████████| 12092/12092 [00:14<00:00, 841.76it/s]


In [4]:
# Custom Dataset Class
class CYP2C9Dataset(Dataset):
    def __init__(self, data, transform=None, mode='1D'):
        self.data = data
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        smiles, label = row['Drug'], row['Y']

        if self.mode == '1D':
            inputs = self.transform(smiles)
            return {key: torch.tensor(val) for key, val in inputs.items()}, torch.tensor(label, dtype=torch.float)

        elif self.mode == 'mhg-gnn':
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            node_features, hyperedges, hyperedge_features = self.transform(mol)
            return {
                'node_features': node_features,
                'hyperedges': hyperedges,
                'hyperedge_features': hyperedge_features
            }, torch.tensor(label, dtype=torch.float)

        elif self.mode == '3D':
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            mol = Chem.AddHs(mol)
            coords = generate_3D_coordinates(mol)
            return torch.tensor(coords, dtype=torch.float), torch.tensor(label, dtype=torch.float)

In [5]:
# 1D: ChemBERTa 데이터 전처리: SMILES 문자열을 ChemBERTa로 변환
tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

def chemberta_transform(smiles):
    return tokenizer(smiles, max_length=128, padding='max_length', truncation=True, return_tensors="pt")

In [6]:
# 2D: mhg-gnn 데이터 전처리: RDKit로 인접 행렬과 원자 특성 생성
def molecular_graph_transform(mol):
    num_atoms = mol.GetNumAtoms()
    node_features = np.array([atom.GetAtomicNum() for atom in mol.GetAtoms()])
    node_features = torch.tensor(node_features, dtype=torch.float).unsqueeze(-1)

    hyperedges = []
    hyperedge_features = []

    for bond in mol.GetBonds():
        atom1, atom2 = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        hyperedges.append([atom1, atom2])
        hyperedge_features.append(bond.GetBondTypeAsDouble())

    hyperedges = torch.tensor(hyperedges, dtype=torch.long)
    hyperedge_features = torch.tensor(hyperedge_features, dtype=torch.float).unsqueeze(-1)

    return node_features, hyperedges, hyperedge_features

In [7]:
# 3D: SchNet 데이터 전처리: RDKit로 3D 좌표 생성
def generate_3D_coordinates(mol):
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    conf = mol.GetConformer()
    coords = np.array([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
    return coords

In [8]:
# collate_fn 정의
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    return torch.utils.data.default_collate(batch)

In [15]:
def collate_fn_2D(batch):
    batch = [b for b in batch if b is not None]  # None 데이터 필터링
    node_features = [item[0]['node_features'] for item in batch]
    hyperedges = [item[0]['hyperedges'] for item in batch]
    hyperedge_features = [item[0]['hyperedge_features'] for item in batch]
    labels = torch.tensor([item[1] for item in batch], dtype=torch.float)

    return {
        'node_features': node_features,
        'hyperedges': hyperedges,
        'hyperedge_features': hyperedge_features,
    }, labels

In [19]:
def collate_fn_3D(batch):
    batch = [b for b in batch if b is not None]  # None 데이터 필터링
    coords = [item[0] for item in batch]  # 각 분자의 좌표
    labels = torch.tensor([item[1] for item in batch], dtype=torch.float)  # 라벨
    return coords, labels

In [10]:
# DataLoader 생성
batch_size = 32

In [11]:
# 1D DataLoader
train_loader_1D = DataLoader(
    CYP2C9Dataset(train_data, transform=chemberta_transform, mode='1D'),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

In [16]:
# 2D DataLoader
train_loader_2D = DataLoader(
    CYP2C9Dataset(train_data, transform=molecular_graph_transform, mode='mhg-gnn'),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn_2D
)

In [20]:
# 3D DataLoader
train_loader_3D = DataLoader(
    CYP2C9Dataset(train_data, transform=None, mode='3D'),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn_3D
)

In [22]:
# 1D 데이터 확인
for batch in train_loader_1D:
    inputs, labels = batch

    # 요약 출력
    print("1D Inputs (Keys):", list(inputs.keys()))  # 입력 데이터 키
    print("1D Input Shape [input_ids]:", inputs['input_ids'].shape)  # 'input_ids' 크기 출력
    print("1D Labels:", labels[:5].tolist(), "...")  # 라벨의 앞 5개만 출력
    break

# 2D 데이터 확인
for batch in train_loader_2D:
    inputs, labels = batch

    # 요약 출력
    print("2D Node Features (First 2):", [nf.shape for nf in inputs['node_features'][:2]])  # 노드 특성 크기
    print("2D Hyperedges (First 2):", [he.shape for he in inputs['hyperedges'][:2]])  # 하이퍼엣지 크기
    print("2D Hyperedge Features (First 2):", [hf.shape for hf in inputs['hyperedge_features'][:2]])  # 하이퍼엣지 특성 크기
    print("2D Labels (First 5):", labels[:5].tolist(), "...")
    break

# 3D 데이터 확인
for batch in train_loader_3D:
    coords, labels = batch

    # 요약 출력
    print("3D Coordinates (Shapes):", [c.shape for c in coords[:5]])  # 첫 5개의 크기 출력
    print("3D Labels (First 5):", labels[:5].tolist(), "...")
    break

1D Inputs (Keys): ['input_ids', 'attention_mask']
1D Input Shape [input_ids]: torch.Size([32, 1, 128])
1D Labels: [0.0, 1.0, 1.0, 1.0, 1.0] ...
2D Node Features (First 2): [torch.Size([23, 1]), torch.Size([25, 1])]
2D Hyperedges (First 2): [torch.Size([25, 2]), torch.Size([28, 2])]
2D Hyperedge Features (First 2): [torch.Size([25, 1]), torch.Size([28, 1])]
2D Labels (First 5): [0.0, 0.0, 0.0, 0.0, 1.0] ...
3D Coordinates (Shapes): [torch.Size([34, 3]), torch.Size([68, 3]), torch.Size([41, 3]), torch.Size([47, 3]), torch.Size([46, 3])]
3D Labels (First 5): [0.0, 0.0, 1.0, 1.0, 1.0] ...


### 1D 데이터 (ChemBERTa)

**Inputs (Keys):**
- `input_ids`: SMILES 문자열을 ChemBERTa 모델에서 사용 가능한 토큰화된 텐서.
- `attention_mask`: 패딩된 토큰을 무시하도록 마스크를 제공하는 텐서.

**Input Shape [input_ids]:** `torch.Size([32, 1, 128])`
- 배치 크기(`batch_size`)는 32.
- 각 샘플은 `1 x 128` 크기이며, 여기서 `128`은 ChemBERTa의 최대 토큰 길이(`max_length`).

**Labels:** `[0.0, 1.0, 1.0, 1.0, 1.0] ...`
- 앞 5개의 라벨 값을 보여줌. 이 라벨은 `0.0` 또는 `1.0`으로 이진 분류임을 나타냄.

---

### 2D 데이터 (mhg-gnn)

**Node Features (First 2):** `[torch.Size([23, 1]), torch.Size([25, 1])]`
- 첫 번째 샘플: 분자에 23개의 노드(원자)가 있으며, 각 노드에는 1개의 특성(예: 원자 번호)이 있음.
- 두 번째 샘플: 분자에 25개의 노드가 있음.

**Hyperedges (First 2):** `[torch.Size([25, 2]), torch.Size([28, 2])]`
- 첫 번째 샘플: 분자에 25개의 결합(하이퍼엣지)이 있으며, 각 결합은 두 노드(원자)의 인덱스를 연결.
- 두 번째 샘플: 분자에 28개의 결합이 있음.

**Hyperedge Features (First 2):** `[torch.Size([25, 1]), torch.Size([28, 1])]`
- 첫 번째 샘플: 각 하이퍼엣지에 대해 1개의 특성(예: 결합 타입)이 있음.
- 두 번째 샘플: 28개의 하이퍼엣지 각각에 1개의 특성이 있음.

**Labels (First 5):** `[0.0, 0.0, 0.0, 0.0, 1.0] ...`
- 앞 5개의 라벨 값을 보여줌. 이 값은 `0.0` 또는 `1.0`으로 이진 분류 라벨임을 나타냄.

---

### 3D 데이터 (SchNet)

**Coordinates (Shapes):** `[torch.Size([34, 3]), torch.Size([68, 3]), torch.Size([41, 3]), torch.Size([47, 3]), torch.Size([46, 3])]`
- 각 분자의 3D 좌표를 나타냄.
- 첫 번째 샘플: 34개의 원자(`[34]`), 각 원자의 3D 좌표는 `[x, y, z]`로 표현되어 3개의 값을 가짐.
- 두 번째 샘플: 68개의 원자.
- 나머지 샘플들도 각각의 원자 수에 따라 좌표 배열이 생성됨.

**Labels (First 5):** `[0.0, 0.0, 1.0, 1.0, 1.0] ...`
- 앞 5개의 라벨 값을 보여줌. 이 값도 이진 분류 라벨임을 나타냄.